In [ ]:
import adi
import pandas as pd
import numpy as np
import os

In [ ]:
path = r'..\uploaded_files\archivo_de_prueba.adicht'


f = adi.read_file(path)
f

file_loaded: True
          h: <cdata 'struct ADI_FileHandle__ * *' owning 8 bytes>
  n_records: 3
 n_channels: 14
    records: Type::list, Len: 3
   channels: Type::list, Len: 14

In [48]:
for ch in f.channels:
    n_samples = len(ch.get_data(2))
    fs_ch = ch.fs[2]
    print(f"{ch.name.strip()}: n_samples/fs = {n_samples}")

BioAmp raw: n_samples/fs = 6462000
HR: n_samples/fs = 64620
ECG: n_samples/fs = 646200
FBP: n_samples/fs = 64620
Valsalva: n_samples/fs = 64620
CO: n_samples/fs = 64620
SV: n_samples/fs = 64620
SVR: n_samples/fs = 64620
ETCO2: n_samples/fs = 64620
SPO2: n_samples/fs = 64620
MCA-L: n_samples/fs = 64620
MCA-R: n_samples/fs = 64620
Tilt Angle: n_samples/fs = 64620
MAP: n_samples/fs = 64620


In [32]:
f.channels[2].get_data(2)

array([0.014375 , 0.0096875, 0.0078125, ..., 0.0553125, 0.055625 ,
       0.0803125], dtype=float32)

In [ ]:
for i in range(f.n_channels):
    print(f.channels[i].name)
    

BioAmp raw
HR
ECG
FBP
Valsalva
CO
SV
SVR
ETCO2
SPO2
MCA-L
MCA-R
Tilt Angle
MAP


In [51]:
f = adi.read_file(path)
channel_names = [ch.name.strip() for ch in f.channels][1:]
channel_data = [ch.get_data(2) for ch in f.channels][1:]
# Usar la frecuencia de muestreo del primer canal (registro 2)
if len(f.channels) > 1:
    fs = f.channels[1].fs[2]
    max_len = max(len(data) for data in channel_data)
    channel_data_padded = [
        np.pad(data, (0, max_len - len(data)), constant_values=np.nan)
        for data in channel_data
    ]
    # Generar columna Time usando fs
    time = np.arange(max_len) / fs
    df_channels = pd.DataFrame(
        {name: data for name, data in zip(channel_names, channel_data_padded)}
    )
    df_channels["Time"] = time
    cols = ["Time"] + [c for c in df_channels.columns if c != "Time"]
    df_channels = df_channels[cols]

df_channels

,Time,HR,ECG,FBP,Valsalva,CO,SV,SVR,ETCO2,SPO2,MCA-L,MCA-R,Tilt Angle,MAP
0,0.00,210.03125,0.014375,78.96875,0.18750,6.400000,121.050003,580.378723,111.292191,99.718124,0.01875,0.00375,0.15625,2.285000
1,0.01,209.90625,0.009688,78.09375,0.15625,6.403125,121.062500,581.210022,111.323128,99.766251,0.02250,-0.01500,0.15625,2.284688
2,0.02,210.15625,0.007812,77.62500,0.12500,6.406250,121.050003,580.544983,111.233749,99.735313,0.03000,0.00375,-0.21875,2.284062
3,0.03,210.00000,0.007500,77.53125,0.18750,6.400000,121.074997,580.711243,111.275002,99.683746,0.01125,0.00375,0.03125,2.284688
4,0.04,210.09375,-0.005937,77.84375,0.12500,6.403125,121.050003,580.711243,111.305939,99.724998,0.01875,-0.00750,-0.06250,2.285312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
646195,6461.95,NaN,0.055312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
646196,6461.96,NaN,0.055312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
646197,6461.97,NaN,0.055312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
646198,6461.98,NaN,0.055625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
